# Web Scraping de Departamentos

Este notebook automatiza el proceso de extracción de datos de arriendo de departamentos desde el sitio [PortalInmobiliario](https://www.portalinmobiliario.com/) para las ciudades de Arica, Iquique, Antofagasta y Calama.

---

## 2. Configuración del entorno

Instalación e importación de las librerías necesarias para realizar el scraping y manipular los datos.

In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

## 3. Inspección del sitio web

Para extraer la información, Identificamos los siguientes selectores CSS:

- **Contenedor de cada publicación:** `'li.ui-search-layout__item'`
- **Nombre:** `'a.poly-component__title'`
- **Precio:** `'div.poly-price__current'`
- **Atributos (dormitorios, baños, metros útiles):** `'ul.poly-attributes_list li.poly-attributes_list__item'`
- **Dirección:** `'span.poly-component__location'`


In [2]:
LISTING_SELECTOR = 'li.ui-search-layout__item'
NAME_SELECTOR    = 'a.poly-component__title'
PRICE_SELECTOR   = 'div.poly-price__current'
ATTRS_SELECTOR   = 'ul.poly-attributes_list li.poly-attributes_list__item'
ADDRESS_SELECTOR = 'span.poly-component__location'

## 4. Descarga de páginas HTML

Se define una función `fetch_page(url)` que descarga el contenido HTML de una página y verifica su estado.

También se prueba con una URL base de búsqueda para verificar que la descarga fue exitosa.

In [3]:
def fetch_page(url):
    """
    Descarga el HTML de la URL indicada.
    Retorna el texto HTML o None si hay un error.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except Exception as e:
        print(f"Error al descargar {url}: {e}")
        return None

# URL de la primera página
base_url = 'https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca'

# Probar la descarga
html = fetch_page(base_url)
if html:
    print('HTML descargado correctamente. Longitud:', len(html))
else:
    print('No se pudo descargar el HTML.')


✅ HTML descargado correctamente. Longitud: 1022269


## 5. Extracción de datos de una página

Se implementa la función `parse_listings(html)` que toma el HTML y extrae los siguientes campos para cada departamento:

- Nombre
- Precio
- Dormitorios
- Baños
- Metros útiles
- Dirección

La función devuelve una lista de diccionarios con los valores extraídos. Además, se incluyen mensajes de advertencia si algún dato no es encontrado.

In [4]:
def parse_listings(html):
    """
    Toma el HTML de una página de resultados y extrae los datos de cada departamento.
    Retorna una lista de dicts con las claves:
        'nombre', 'precio', 'dormitorios', 'baños', 'metros', 'direccion'.
    Además imprime advertencias si algún selector no encuentra nada.
    """
    soup = BeautifulSoup(html, 'html.parser')
    containers = soup.select(LISTING_SELECTOR)
    print(f"Encontrados {len(containers)} contenedores con LISTING_SELECTOR")
    results = []
    
    for idx, c in enumerate(containers, 1):
        # Nombre
        title_elem = c.select_one(NAME_SELECTOR)
        nombre = title_elem.get_text(strip=True) if title_elem else None
        if not title_elem:
            print(f"Contenedor {idx}: NO se encontró el nombre usando {NAME_SELECTOR}")
        
        # Precio
        price_elem = c.select_one(PRICE_SELECTOR)
        precio = price_elem.get_text(strip=True) if price_elem else None
        if not price_elem:
            print(f"Contenedor {idx}: NO se encontró el precio usando {PRICE_SELECTOR}")
        
        # Atributos: dormitorios, baños, metros
        attrs = c.select(ATTRS_SELECTOR)
        if len(attrs) < 3:
            print(f"Contenedor {idx}: se encontraron {len(attrs)} elementos con {ATTRS_SELECTOR} (esperaba ≥3)")
        dormitorios = attrs[0].get_text(strip=True) if len(attrs) > 0 else None
        baños       = attrs[1].get_text(strip=True) if len(attrs) > 1 else None
        metros      = attrs[2].get_text(strip=True) if len(attrs) > 2 else None
        
        # Dirección
        addr_elem = c.select_one(ADDRESS_SELECTOR)
        direccion = addr_elem.get_text(strip=True) if addr_elem else None
        if not addr_elem:
            print(f"Contenedor {idx}: NO se encontró la dirección usando {ADDRESS_SELECTOR}")
        
        results.append({
            'nombre':      nombre,
            'precio':      precio,
            'dormitorios': dormitorios,
            'baños':       baños,
            'metros':      metros,
            'direccion':   direccion
        })
    
    return results

deptos = parse_listings(html)
print(f"\nSe extrajeron {len(deptos)} registros.")
deptos[:3]

🔍 Encontrados 48 contenedores con LISTING_SELECTOR
⚠️ Contenedor 33: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
⚠️ Contenedor 36: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)

✅ Se extrajeron 48 registros.


[{'nombre': 'Arriendo Departamento Amoblado 3 Habitaciones En Iquique',
  'precio': '$850.000',
  'dormitorios': '3 dormitorios',
  'baños': '2 baños',
  'metros': '84 m² útiles',
  'direccion': 'Av. 4 Sur 2400 - 2700, Iquique, Cerro Dragón, Iquique, Tarapacá'},
 {'nombre': 'Vista Al Mar !! Gastos Comunes Incluidos !! 3 D 2 B Est  Bod',
  'precio': '$700.000',
  'dormitorios': '3 dormitorios',
  'baños': '2 baños',
  'metros': '90 m² útiles',
  'direccion': 'Eleuterio Ramirez 1559, Iquique, Centro De Iquique, Iquique, Tarapacá'},
 {'nombre': 'Eleuterio Ramirez- Gastos Comunes Incluidos  3d 2b  1 Est 1b',
  'precio': '$700.000',
  'dormitorios': '3 dormitorios',
  'baños': '2 baños',
  'metros': '90 m² útiles',
  'direccion': 'Eleuterio Ramirez 1559, Iquique, Centro De Iquique, Iquique, Tarapacá'}]

## 6. Paginación automática

Se implementa un mecanismo para iterar automáticamente por las distintas páginas de resultados, concatenando los datos de cada una en una sola lista.

In [5]:
def get_next_page_url(html, current_url):
    """Extrae la URL de la página siguiente."""
    soup = BeautifulSoup(html, 'html.parser')
    # 1) <link rel="next"> en el <head>
    link_next = soup.find('link', rel='next')
    if link_next and link_next.get('href'):
        return urljoin(current_url, link_next['href'])
    # 2) Botón 'Siguiente' con clase específica (andes-pagination__link)
    a_next = soup.select_one('a.andes-pagination__link[title="Siguiente"]')
    if a_next and a_next.get('href'):
        return urljoin(current_url, a_next['href'])
    # 3) Fallback: li que contiene el botón next
    btn = soup.select_one('li.andes-pagination__button--next a')
    if btn and btn.get('href'):
        return urljoin(current_url, btn['href'])
    # No hay más páginas
    return None


def scrape_all_pages_auto(base_url, pause=1.0):
    url = base_url
    all_results = []
    page = 1
    while url:
        print(f"🔄 Página {page}: {url}")
        html = fetch_page(url)
        if not html:
            break
        results = parse_listings(html)
        if not results:
            break
        all_results.extend(results)
        next_url = get_next_page_url(html, url)
        if not next_url:
            print("Fin de páginas.")
            break
        time.sleep(pause)
        url = next_url
        page += 1
    return all_results

## 7. Scraping de múltiples ciudades

Se realiza scraping para distintas ciudades del norte de Chile. Los resultados se combinan y luego se exportan a un archivo `.csv` para su análisis posterior.

In [6]:
cities = {
    'arica':       'https://www.portalinmobiliario.com/arriendo/departamento/arica-arica-y-parinacota',
    'iquique':     'https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca/_NoIndex_True',
    'antofagasta': 'https://www.portalinmobiliario.com/arriendo/departamento/antofagasta-antofagasta',
    'calama':      'https://www.portalinmobiliario.com/arriendo/departamento/calama-antofagasta'
}

all_data = []
for city, url in cities.items():
    print(f"\nScraping {city}...")
    rec = scrape_all_pages_auto(url)
    for r in rec:
        r['ciudad'] = city
    all_data.extend(rec)

# Construir DataFrame y revisar nulos
df = pd.DataFrame(all_data)
print(df.isna().sum())


🏙️ Scraping arica...
🔄 Página 1: https://www.portalinmobiliario.com/arriendo/departamento/arica-arica-y-parinacota
🔍 Encontrados 31 contenedores con LISTING_SELECTOR
⚠️ Contenedor 5: se encontraron 2 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
✅ Fin de páginas.

🏙️ Scraping iquique...
🔄 Página 1: https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca/_NoIndex_True
🔍 Encontrados 48 contenedores con LISTING_SELECTOR
⚠️ Contenedor 33: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
⚠️ Contenedor 36: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
✅ Fin de páginas.

🏙️ Scraping antofagasta...
🔄 Página 1: https://www.portalinmobiliario.com/arriendo/departamento/antofagasta-antofagasta
🔍 Encontrados 48 contenedores con LISTING_SELECTOR
✅ Fin de páginas.

🏙️ Scraping calama...
🔄 Página 1: https://www.portalinmobiliario.com/arriendo/dep

## 8. Scraping con Selenium

En esta sección se utiliza **Selenium** para automatizar la navegación:

1. Renderizar contenido generado por JavaScript.
2. Hacer clic en el botón "Siguiente" para avanzar entre páginas.

In [7]:
def scrape_with_selenium_click(base_url, pause=2.0, headless=True):
    """Scrapea con Selenium desplazándose hasta el botón 'Siguiente' y clickeándolo."""
    options = webdriver.ChromeOptions()
    if headless:
        options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)')

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    driver.get(base_url)
    time.sleep(2)

    # Eliminar banner de cookies si existe
    try:
        banner = driver.find_element(By.ID, 'newCookieDisclaimerBanner')
        driver.execute_script("arguments[0].style.display = 'none';", banner)
        print("Banner de cookies eliminado.")
    except:
        pass

    all_data = []
    page = 1
    while True:
        # Esperar a que cargue al menos un resultado
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, LISTING_SELECTOR))
        )
        print(f"Página {page} cargada: {driver.current_url}")

        # Parsear listados
        html = driver.page_source
        registros = parse_listings(html)
        if not registros:
            print("No se extrajeron registros, deteniendo.")
            break
        all_data.extend(registros)
        print(f"Selenium Click Página {page}: {len(registros)} registros")

        # Scroll hasta el botón Siguiente y click
        try:
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[title="Siguiente"]'))
            )
            # Asegurarse de quitar banner en cada página antes de click
            try:
                banner = driver.find_element(By.ID, 'newCookieDisclaimerBanner')
                driver.execute_script("arguments[0].style.display = 'none';", banner)
            except:
                pass
            driver.execute_script("arguments[0].scrollIntoView(true);", next_btn)
            time.sleep(1)
            next_btn.click()
            print("Click en Siguiente realizado.")
            page += 1
            time.sleep(pause)
        except Exception as e:
            print(f"No se pudo avanzar: {e}")
            break

    driver.quit()
    return all_data

In [8]:
df_selenium = pd.DataFrame(scrape_with_selenium_click(cities['iquique'], headless=False))
print(f"Total Iquique (Selenium Click): {len(df_selenium)}")

📢 Banner de cookies eliminado.
📄 Página 1 cargada: https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca/_NoIndex_True
🔍 Encontrados 48 contenedores con LISTING_SELECTOR
⚠️ Contenedor 33: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
⚠️ Contenedor 36: se encontraron 1 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
✅ Selenium Click Página 1: 48 registros
⏭️ Click en Siguiente realizado.
📄 Página 2 cargada: https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca/_Desde_49_NoIndex_True
🔍 Encontrados 48 contenedores con LISTING_SELECTOR
⚠️ Contenedor 8: se encontraron 2 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
✅ Selenium Click Página 2: 48 registros
⏭️ Click en Siguiente realizado.
📄 Página 3 cargada: https://www.portalinmobiliario.com/arriendo/departamento/iquique-tarapaca/_Desde_97_NoIndex_True
🔍 Encontrados 48 contenedores

## 9. Scraping de múltiples ciudades con Selenium

Se aplica la técnica de Selenium a varias ciudades, obteniendo los datos de cada una de forma dinámica. Finalmente, se exportan todos los resultados a un archivo `.csv`.

In [9]:
todos_selenium = []
for city, url in cities.items():
    print(f"🗺️ Scraping Selenium para {city.title()}")
    registros = scrape_with_selenium_click(url, pause=2, headless=True)
    for r in registros:
        r['ciudad'] = city
    todos_selenium.extend(registros)

# 8.2 Crear DataFrame y revisar
df_all = pd.DataFrame(todos_selenium)
print("Resumen de registros combinados:")
print(df_all.isna().sum())
print(f"Total de registros Selenium (todas las ciudades): {df_all.shape[0]}")

# 8.3 Exportar a CSV
data_csv = 'departamentos_arriendo.csv'
df_all.to_csv(data_csv, index=False)
print(f"CSV guardado en: {data_csv}")

🗺️ Scraping Selenium para Arica
📢 Banner de cookies eliminado.
📄 Página 1 cargada: https://www.portalinmobiliario.com/arriendo/departamento/arica-arica-y-parinacota
🔍 Encontrados 31 contenedores con LISTING_SELECTOR
⚠️ Contenedor 5: se encontraron 2 elementos con ul.poly-attributes_list li.poly-attributes_list__item (esperaba ≥3)
✅ Selenium Click Página 1: 31 registros
🚫 No se pudo avanzar: Message: 
Stacktrace:
	GetHandleVerifier [0x0x9c3b03+62899]
	GetHandleVerifier [0x0x9c3b44+62964]
	(No symbol) [0x0x7f10f3]
	(No symbol) [0x0x83980e]
	(No symbol) [0x0x839bab]
	(No symbol) [0x0x8825c2]
	(No symbol) [0x0x85e554]
	(No symbol) [0x0x87fd81]
	(No symbol) [0x0x85e306]
	(No symbol) [0x0x82d670]
	(No symbol) [0x0x82e4e4]
	GetHandleVerifier [0x0xc24793+2556483]
	GetHandleVerifier [0x0xc1fd02+2537394]
	GetHandleVerifier [0x0x9ea2fa+220586]
	GetHandleVerifier [0x0x9daae8+157080]
	GetHandleVerifier [0x0x9e141d+184013]
	GetHandleVerifier [0x0x9cba68+95512]
	GetHandleVerifier [0x0x9cbc10+95936]
	

## 10. Limpieza de la columna `precio`

Se realiza la limpieza de datos en la columna de precios:

- Conversión de precios en UF a CLP utilizando el valor actual de la UF.
- Eliminación de puntos, símbolos `$` y la palabra `UF`.
- Conversión del resultado a número entero redondeado.

In [10]:
import re

# Ajustar valor al de la UF vigente
UF_VALUE = 39240.93  

# limpiar precios
def clean_price(s):
    txt = str(s).upper().replace('.', '').replace('$', '').replace(' ', '')
    if 'UF' in txt:
        num = float(re.search(r'(\d+)', txt).group(1))
        return int(round(num * UF_VALUE, 0))
    return int(round(float(re.sub(r'[^\d]', '', txt)), 0))

# limpieza de precio
df_all['precio'] = df_all['precio'].apply(clean_price)

# extraer y limpiar dormitorios y baños e imputar moda por ciudad
for col in ['dormitorios', 'baños']:
    num_col = col + '_num'
    df_all[num_col] = df_all[col].str.extract(r'(\d+)').astype(float)
    df_all[num_col] = df_all.groupby('ciudad')[num_col].transform(
        lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.median())
    )

# Extraer metros e imputar mediana por ciudad y dormitorios
df_all['metros_num'] = df_all['metros'].str.extract(r'(\d+)').astype(float)
median_m = df_all.groupby(['ciudad', 'dormitorios_num'])['metros_num'].median()

def fill_metros(row):
    if pd.isna(row['metros_num']):
        return median_m.loc[(row['ciudad'], row['dormitorios_num'])]
    return row['metros_num']

df_all['metros_num'] = df_all.apply(fill_metros, axis=1)

# Seleccionar columnas finales y convertir a enteros
cols_final = ['nombre', 'direccion', 'ciudad', 'precio', 'dormitorios_num', 'baños_num', 'metros_num']
df_clean = df_all[cols_final].copy()
df_clean.columns = ['nombre', 'direccion', 'ciudad', 'precio', 'dormitorios', 'baños', 'metros']
# Eliminar filas que todavía tengan algún NaN en las columnas numéricas
df_clean = df_clean.dropna(subset=['precio','dormitorios','baños','metros'])

# Ahora ya podemos convertir seguro a int
for c in ['precio','dormitorios','baños','metros']:
    df_clean[c] = df_clean[c].astype(int)

# Exportar CSV limpio
df_clean.to_csv('departamentos_arriendo_limpio.csv', index=False, encoding='utf-8')

## Cálculo de precio por m²

A continuación calculamos una nueva columna precio_por_m2, redondeada a 0 decimales, a partir del precio y metros_num.

In [11]:
df_m2 = pd.read_csv('departamentos_arriendo_limpio.csv')

df_m2['precio_m2'] = round(df_m2['precio'] / df_m2['metros'], 0)

for m in ['precio_m2']:
    df_m2[m] = df_m2[m].astype(int)

df_m2.to_csv('departamentos_arriendo_limpio.csv', index=False, encoding='utf-8')